# DSCI100 Final Project Report: Predicting Subscription of Newsletter for a Video Game Research Server

Names: Christopher Lee, ...

## 1) Introduction

The research group led by UBC computer science professor Frank Wood set up a Minecraft server to collect data to learn more about people's video game behaviours. Demographics such as age, gender, and experience as well as the number of hours played on the server were tracked. A major challenge in running this project is ensuring how the team should target recruitment efforts. Therefore, this data analysis was done as an exploration in part with Dr. Wood's team. Our analysis will focus on one of the major issues related to targeting recruitment efforts, which is focusing on certain behaviours or characteristics in predicting subscription to a newsletter. 

In this project, we aimed to respond to this issue by exploring the question: 

**"Can age and ____ (played hours)?? predict whether or not a player will subscribe to the newsletter?"**

We attempted to answer this question by employing a predictive K-NN classfication model. Data from the "players.csv" dataset was used as it included the relevant predictor and response variables. This dataset includes 196 observations and 7 variables.

Variable information for the "players.csv" dataset:
1. `experience` (**character**):
    - player's experience playing Minecraft
2. `subscribe` (**logical**):
    - player's subscription status
3. `hashedEmail` (**character**):
    - player's hashed email
4. `played_hours` (**numeric (double)**):
    - player's total played hours on the server
5. `name` (**character**):
    - player's name
6. `gender` (**character**):
    - player's gender
7. `Age` (**numeric (double)**):
    - player's age

The dataset may include potential issues, such as NA values for some players' ages.

## 2) Methods & Results